In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
from itertools import combinations

# Предобработка

In [2]:
answers = {} # make dictionary for answers

def general_names(names):
    """Make same name for WB and Universal subcompany"""
    company = []
    for name in names.split('|'):
        if "Warner Bros" in name:
            company.append("Warner Bros")
        elif "Universal" in name:
            company.append("Universal Pictures")
        else:
            company.append(name)
    return company

data = pd.read_csv('movie_bd_v5.csv')

# Split and sort data for ease use latter
data['genres'] = data['genres'].apply(lambda x:x.split('|'))
data['director'] = data['director'].apply(lambda x:x.split('|'))
data['cast'] = data['cast'].apply(lambda x:sorted(x.split('|')))

data['production_companies'] = data['production_companies'].apply(general_names)

# Add and calculete date for futere use
data['month'] = data['release_date'].apply(lambda x:int(x.split('/')[0]))
data['profit'] = data.revenue - data.budget
data['name_len'] = data['original_title'].apply(lambda x:len(x))
data['overview_words'] = data['overview'].apply(lambda x:len(x.split(' ')))

# 1. У какого фильма из списка самый большой бюджет?

In [3]:
answers['1'] = data[data.budget == data.budget.max()].original_title.iloc[0]

# 2. Какой из фильмов самый длительный (в минутах)?

In [4]:
answers['2'] = data[data.runtime == data.runtime.max()].original_title.iloc[0]

# 3. Какой из фильмов самый короткий (в минутах)?





In [5]:
answers['3'] = data[data.runtime == data.runtime.min()].original_title.iloc[0]

# 4. Какова средняя длительность фильмов?


In [6]:
answers['4'] = round(data.runtime.mean())

# 5. Каково медианное значение длительности фильмов? 

In [7]:
answers['5'] = round(data.runtime.median())

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [8]:
answers['6'] = data[data.profit == data.profit.max()].original_title.iloc[0]

# 7. Какой фильм самый убыточный? 

In [9]:
answers['7'] = data[data.profit == data.profit.min()].original_title.iloc[0]

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [10]:
answers['8'] = data[data.profit > 0].original_title.count()

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [11]:
data_temp = data[data.release_year == 2008]
answers['9'] = data_temp[data_temp.profit == data_temp.profit.max()].original_title.iloc[0]

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [12]:
data_temp = data.query('2012 <= release_year <= 2014')
answers['10'] = data_temp[data_temp.profit == data_temp.profit.min()].original_title.iloc[0]

# 11. Какого жанра фильмов больше всего?

In [13]:
c = Counter()
for genre in data.genres:
    for x in genre:
        c[x] += 1
answers['11'] = c.most_common(1)[0][0]

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [14]:
c_profit = Counter()
for genre in data.genres[data.profit>0]:
    for x in genre:
        c_profit[x] += 1
answers['12'] = c_profit.most_common(1)[0][0]

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [15]:
answers['13'] = data.explode('director').groupby(['director'])['revenue'].sum().sort_values(ascending=False).axes[0][0]

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [16]:
answers['14'] = data.explode('genres').query('genres == "Action"').explode('director').groupby(['director'])['original_title'].count().sort_values(ascending=False).axes[0][0]

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [17]:
answers['15'] = data[data.release_year == 2012].explode('cast').groupby(['cast'])['revenue'].sum().sort_values(ascending=False).axes[0][0]

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [18]:
answers['16'] = data[data.budget > data.budget.mean()].explode('cast').groupby(['cast'])['original_title'].count().sort_values(ascending=False).axes[0][0]

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [19]:
answers['17'] = data.explode('cast').query('cast == "Nicolas Cage"').explode('genres').groupby(['genres'])['original_title'].count().sort_values(ascending=False).axes[0][0]

# 18. Самый убыточный фильм от Paramount Pictures

In [20]:
answers['18'] = data.explode('production_companies').query('production_companies == "Paramount Pictures"').query('profit == profit.min()')['original_title'].iloc[0]

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [21]:
answers['19'] = data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False).axes[0][0]

# 20. Какой самый прибыльный год для студии Warner Bros?

In [22]:
all_company = data.explode('production_companies')
answers['20'] = all_company[all_company.production_companies.str.contains("Warner Bros")].groupby(['release_year'])['profit'].sum().sort_values(ascending=False).axes[0][0]

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [23]:
answers['21'] = data.groupby(['month'])['original_title'].count().sort_values(ascending=False).axes[0][0]

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [24]:
answers['22'] = len(data.query('5 < month < 9'))

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [25]:
answers['23'] = data.query('month in [12,1,2]').explode('director').groupby(['director'])['original_title'].count().sort_values(ascending=False).axes[0][0]

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [26]:
answers['24'] = data.explode('production_companies')[data.explode('production_companies').name_len == data.name_len.max()].production_companies.tolist()

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [27]:
answers['25'] = data.explode('production_companies').groupby(['production_companies'])['overview_words'].mean().sort_values(ascending=False).axes[0][0]

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [28]:
answers['26'] = data.sort_values('vote_average', ascending=False).original_title.head(int(len(data)*0.01)).tolist()

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [29]:
pairs = {}

def find_pairs(cast):
    for x in combinations(cast, 2):
        if x in pairs:
            pairs[x] += 1
        else:
            pairs[x] = 1

data.cast.apply(find_pairs)
max_val = max(pairs.values())
answers['27'] = {k:v for k, v in pairs.items() if v == max_val}

# Submission

In [30]:
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides',
 '2': 'Gods and Generals',
 '3': 'Winnie the Pooh',
 '4': 110,
 '5': 107,
 '6': 'Avatar',
 '7': 'The Lone Ranger',
 '8': 1478,
 '9': 'The Dark Knight',
 '10': 'The Lone Ranger',
 '11': 'Drama',
 '12': 'Drama',
 '13': 'Peter Jackson',
 '14': 'Robert Rodriguez',
 '15': 'Chris Hemsworth',
 '16': 'Matt Damon',
 '17': 'Action',
 '18': 'K-19: The Widowmaker',
 '19': 2015,
 '20': 2014,
 '21': 9,
 '22': 450,
 '23': 'Peter Jackson',
 '24': ['Twentieth Century Fox Film Corporation', 'Four By Two Productions'],
 '25': 'Midnight Picture Show',
 '26': ['The Dark Knight',
  'Interstellar',
  'The Imitation Game',
  'Inside Out',
  'Room',
  'The Wolf of Wall Street',
  'Gone Girl',
  '12 Years a Slave',
  'Guardians of the Galaxy',
  'The Lord of the Rings: The Return of the King',
  'Memento',
  'Inception',
  'The Pianist',
  'The Grand Budapest Hotel',
  'Her',
  'Spotlight',
  'Big Hero 6',
  'The Fault in Our Stars'],
 '27': {('Daniel Radclif